In [28]:
# Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
from census import Census
import gmaps

# Census API Keys
from config import (api_key)
c = Census(api_key, year=2017)

### The poverty numbers in the dataframe is "Number of persons whose income in the past 12 months is below the poverty level"

In [29]:
#"B25077_001E": "Median Home Value"

census_data = c.acs5.get(("NAME",
                          "B01003_001E",
                          "B17001_002E", 
                          "B17001A_002E", "B17001B_002E", 
                          "B17001D_002E", "B17001I_002E"), 
                         {'for': 'county:*','in':'state:19'})

# Convert to DataFrame
census_pd = pd.DataFrame(census_data)

# Column Reordering
census_pd = census_pd.rename(columns={"B01003_001E": "Population",
                                      "B17001_002E": "Poverty Count",
                                      "B17001A_002E": "Poverty White",
                                      "B17001B_002E": "Poverty Black", 
                                      "B17001D_002E": "Poverty Asian", 
                                      "B17001I_002E": "Poverty Hispanic",
                                      "NAME": "Name"})

# Add in Poverty Rate (Poverty Count / Population)
census_pd["Total Poverty Rate (%)"] = 100 * census_pd["Poverty Count"].astype(int) \
                                        / census_pd["Population"].astype(int)
census_pd["White Poverty Rate (%)"] = 100 * census_pd["Poverty White"].astype(int) \
                                        / census_pd["Population"].astype(int)
census_pd["Black Poverty Rate (%)"] = 100 * census_pd["Poverty Black"].astype(int) \
                                        / census_pd["Population"].astype(int)
census_pd["Asian Poverty Rate (%)"] = 100 * census_pd["Poverty Asian"].astype(int) \
                                        / census_pd["Population"].astype(int)
census_pd["Hispanic Poverty Rate (%)"] = 100 * census_pd["Poverty Hispanic"].astype(int) \
                                        / census_pd["Population"].astype(int)


# Final DataFrame
census_pd = census_pd[["Name", "Total Poverty Rate (%)", "White Poverty Rate (%)", 
                       "Black Poverty Rate (%)", "Asian Poverty Rate (%)", 
                       "Hispanic Poverty Rate (%)" ]]



split = census_pd["Name"].str.split(",", n = 1, expand = True)
census_pd["County"]= split[0]
census_pd["State"]=split[1]
census_pd = census_pd[["State", "County", 
                       "Total Poverty Rate (%)", "White Poverty Rate (%)", 
                       "Black Poverty Rate (%)", "Asian Poverty Rate (%)", 
                       "Hispanic Poverty Rate (%)"]]

census_pd = census_pd.set_index("State")
census_pd.mean()
census_pd.to_csv("2017_Race_Data")